# Formation 2022  

##  Initiation à l’analyse de données Minion pour l'analyse de métagénome viraux

### PART 1

Created by J. Orjuela (DIADE-IRD), A. Comte (PHIM-IRD) and D. Filloux (PHIM-CIRAD) from a formation by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) given in Septembre 2021

Septembre 2022

## 1. Creating the folder, downloading data and so on

### 1.1 Data

Before starting, please download special data created for this practical training. Data are available on the from I-Trop server.

This data are pineapple metagenome.... (description++)

In [ ]:
cd ~
mkdir -p SG-ONT-2022/DATA
cd SG-ONT-2022/DATA

# download data already basecalled
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/data.fastq
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/sequencing_summary.txt

### 1.2 Reference Genome

For cleaning the dataset, you need to download the reference genome of the host (pineapple).

These data comme from [NCBI.](https://www.ncbi.nlm.nih.gov/data-hub/genome/GCF_001540865.1/)


In [ ]:
cd ~/SG-ONT-2022/DATA

# download reference genome of the pineapple
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/GCA_001540865.1_Ananas_comosus_cultivar_F153.fasta

## 2. Basecalling 

When DNA molecules are sequenced, electrical signals are stocked on fast5 format files.

These signals need to be converted on standard fastq files to post-analysis.

Several training dataset models are usually used to convert fast5 to fastq. 


### 2.1 Basecalling with Guppy

Guppy is a data processing toolkit that contains the 'Oxford Nanopore Technologies' basecalling algorithms, and several bioinformatic post-processing features.

Basecalling with guppy can be launch using guppy tool. 

Guppy takes fast5 raw read files and transform electrical signal in fastq files.

We recommend to basecaller yours dataset using a GPU graphic card to obtain results quickly.

Documentation about how to run Guppy on I-trop GPU can be found on https://bioinfo.ird.fr/index.php/tutorials-fr/gpu/


In [ ]:
#To see all the documentation of guppy:
  guppy_basecaller --help

To run guppy you need to choose your configuration file accordingly to the flowcell and the kityou used for sequencing.

In [ ]:
#List supported flowcells and kits:
  guppy_basecaller --print_workflows

3 types of config files:
- sup : high accuracy, very slow
- hac : medium accuracy, medium recources needes
- fast : poor accuracy, very fast

#### Basecalling command line <span style="color:red"> (Don't run it! ) </span>

In [ ]:
guppy_basecaller -c dna_r9.4.1_450bps_sup.cfg -i fast5/ -r -s output --num_callers 4 --gpu_runners_per_device 8 --min_qscore 6 --device cuda:2 --trim_adapters --detect_adapter --detect_mid_strand_adapter --pt_scaling --do_read_splitting

## 3. Quality Control on Long Reads


Calculating data quantity

In [ ]:
cd ~/SG-ONT-2022/DATA
pwd

Calculating how many reads are in the fastq file

In [ ]:
awk '{s++}END{print s/4}' data.fastq

Calculating how many bases were sequenced

In [ ]:
seqtk seq -A data.fastq | grep -v ">" |wc -m

What is the sequencing depth?

### 3.1 Quality Control of FASTQ with Nanoplot

Control reads quality using Nanoplot. You can parameter this tool using --help.

In [ ]:
NanoPlot --help

Launch NanoPlot. You can launch NanoPlot using summaries or fastq files.

In [ ]:
# create a folder to save results
mkdir -p ~/SG-ONT-2022/RESULTS
cd ~/SG-ONT-2022/RESULTS

###### run nanoplot 
NanoPlot -t 1 --summary sequencing_summary.txt --outdir NANOPLOT

Check stats on created NanoStats file.

In [ ]:
cat NANOPLOT/NanoStats.txt

* What do you think about data? 

* Estimate coverage.

* What about reads quality qscore?

### 3.2 Quality Control of FASTQ with Nanoplot (Hh real data --fastq )

Check quality using NanoPlot over Hh real data fastq. This can take a while.

In [ ]:
cd ~/SG-ONT-2022/RESULTS/
pwd

**The following Nanoplot will run for a few minutes**

In [ ]:
time NanoPlot -t 1 --fastq ../DATA/real_Hh/H_M1C132_1.fastq.gz --outdir NANOPLOT_Hh

Observe report.html.

* what about this dataset?

### 3.3 Compare reads QC statistics with NanoComp (Summaries dataset)

Compare long reads sequencing datasets using **NanoComp**.

NanoComp compiles quality information in a useful html report.

You can launch NanoComp using summaries or fastq files.

In [ ]:
NanoComp --help


For the moment, we used only a Clone and simulated data. 

Please download a "summaries" obtained in paspalum real data. 

Download available 'Real_PSummaries' to compare them. 


In [ ]:
cd ~/SG-ONT-2021/DATA
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training/real_summaries.tar.gz

In [ ]:
#decompress 
tar -zxvf real_summaries.tar.gz

In [ ]:
cd ~/SG-ONT-2021/DATA/real_summaries/
ls -lh  *txt

Compare this 3 summaries files with NanoComp

In [ ]:
time NanoComp --summary OLD_summary.txt FF_summary.txt BON_summary.txt --outdir ~/SG-ONT-2021/RESULTS/NANOCOMP-realsummaries

In [ ]:
conda deactivate

###  Others complemental tools: 

https://github.com/wdecoster/NanoPlot#companion-scripts

* NanoComp: comparing multiple runs

* NanoStat: statistic summary report of reads or alignments

* NanoFilt: filtering and trimming of reads

* NanoLyse: removing contaminant reads (e.g. lambda control DNA) from fastq

FiltLong : filtering long reads by quality https://github.com/rrwick/Filtlong
